In [14]:
import os
import cv2
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import pandas as pd
from PIL import Image
import onnxruntime
import onnx

In [5]:
def preprocess(image):
    # Resize to currently hardcoded 1344x1344
    image = image.resize((1344, 1344))
    
    # Convert to BGR
    image = np.array(image)[:, :, [2, 1, 0]].astype('float32')

    # HWC -> CHW
    image = np.transpose(image, [2, 0, 1])

    # Pad to be divisible of 32 - Might need this later on if I don't hardcode image size to 1344x1344
    #import math
    #padded_h = int(math.ceil(image.shape[1] / 32) * 32)
    #padded_w = int(math.ceil(image.shape[2] / 32) * 32)

    #padded_image = np.zeros((3, padded_h, padded_w), dtype=np.float32)
    #padded_image[:, :image.shape[1], :image.shape[2]] = image
    #image = padded_image
    
    return image

In [23]:
model_name = '/Users/henrylydecker/Documents/projects/aerial-segmentation/weights/urban_small_poc.onnx'
onnx_model = onnx.load(model_name)
onnx.checker.check_model(onnx_model)

In [ ]:
device = "cpu"
image = Image.open("/Users/henrylydecker/Desktop/test.png")


ort_session = onnxruntime.InferenceSession(model_name)

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

img = Image.open("/Users/henrylydecker/Desktop/test.png")

resize = transforms.Resize([224, 224])
img = resize(img)

img_ycbcr = img.convert('YCbCr')
img_y, img_cb, img_cr = img_ycbcr.split()

to_tensor = transforms.ToTensor()
img_y = to_tensor(img_y)
img_y.unsqueeze_(0)

ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(img_y)}
ort_outs = ort_session.run(None, ort_inputs)
img_out_y = ort_outs[0]

img_out_y = Image.fromarray(np.uint8((img_out_y[0] * 255.0).clip(0, 255)[0]), mode='L')

# get the output image follow post-processing step from PyTorch implementation
final_img = Image.merge(
    "YCbCr", [
        img_out_y,
        img_cb.resize(img_out_y.size, Image.BICUBIC),
        img_cr.resize(img_out_y.size, Image.BICUBIC),
    ]).convert("RGB")

In [9]:
session = onnxruntime.InferenceSession("/Users/henrylydecker/Documents/projects/aerial-segmentation/weights/urban_small_poc.onnx")

image = Image.open("/Users/henrylydecker/Desktop/test.png")

# Preprocess the image and convert to a numpy array of float:
# 1. reinterpolate it to the hardcoded size
# 2. reorder the colour axis as requires by the model (RGB -> BGR)
# 3. reorder the pixel/color array (YXC -> CXY)
img_data = preprocess(image)

# Predict using ONNX Runtime - throw away unneeded outputs (Still trying to work out what the extra outputs actually are)
boxes, labels, _, scores, _ = session.run(None, {
    session.get_inputs()[0].name: img_data
})
    

ValueError: not enough values to unpack (expected 5, got 4)